
This script is used to check your local docker images.

Two methods can be used - shell commands with subprocess. Popen, or
use the docker SDK (docker-py) to manage and inspect local configuration and running images.

A function is declared to wrap executing a command insider an image which retrieves the contract addresses
from the contract artifact JSON files (ABIs). Use this to confirm that your local configuration (as
specified in config_local.ini) matches the deployed images.

In [2]:
import docker
import subprocess
# import subprocess
# import pathlib
# import squid_py.ocean as ocean
import sys
from pprint import pprint

In [3]:
import logging

loggers_dict = logging.Logger.manager.loggerDict

logger = logging.getLogger()
logger.handlers = []

# Set level
logger.setLevel(logging.DEBUG)

# FORMAT = "%(asctime)s - %(levelno)s - %(module)-15s - %(funcName)-15s - %(message)s"
# FORMAT = "%(asctime)s %(levelno)s: %(module)30s %(message)s"
FORMAT = "%(levelno)s - %(module)-15s - %(funcName)-15s - %(message)s"

DATE_FMT = "%Y-%m-%d %H:%M:%S"
DATE_FMT = "%Y-%m-%d %H:%M:%S"
formatter = logging.Formatter(FORMAT, DATE_FMT)

# Create handler and assign
handler = logging.StreamHandler(sys.stderr)
handler.setFormatter(formatter)
logger.handlers = [handler]
logger.debug("Logging started")

10 - <ipython-input-3-f3cf088e1dc8> - <module>        - Logging started


In [4]:
# s = subprocess.check_output('docker ps', shell=True).wait()
s = subprocess.Popen("docker ps" + "", shell=True).wait()
print(s)

0


In [5]:
# High level client
client = docker.from_env()
# Get the APIClient for running commands
low_level_api_client = docker.APIClient(base_url='unix://var/run/docker.sock')

10 - config          - find_config_file - Trying paths: ['/home/batman/.docker/config.json', '/home/batman/.dockercfg']
10 - config          - find_config_file - Found file at path: /home/batman/.docker/config.json
10 - auth            - load_config     - Found 'auths' section
10 - auth            - parse_auth      - Found entry (registry='133126667150.dkr.ecr.eu-central-1.amazonaws.com', username='AWS')
10 - auth            - parse_auth      - Found entry (registry='https://index.docker.io/v1/', username='plasmafrog')
10 - config          - find_config_file - Trying paths: ['/home/batman/.docker/config.json', '/home/batman/.dockercfg']
10 - config          - find_config_file - Found file at path: /home/batman/.docker/config.json
10 - auth            - load_config     - Found 'auths' section
10 - auth            - parse_auth      - Found entry (registry='133126667150.dkr.ecr.eu-central-1.amazonaws.com', username='AWS')
10 - auth            - parse_auth      - Found entry (registry='htt

In [6]:
for container in client.containers.list():
    print(f"Docker container {container.name} is {container.status}")
    print('\tTags:', container.image.tags)
    # print(container.labels)
    # print("\n")
# container.logs()

10 - connectionpool  - _make_request   - http://localhost:None "GET /v1.35/containers/json?limit=-1&all=0&size=0&trunc_cmd=0 HTTP/1.1" 200 None
10 - connectionpool  - _make_request   - http://localhost:None "GET /v1.35/containers/0d109c3461e06e2cf855f4c1e854c635981935fc7a015a36b1c7ef0c33ff267b/json HTTP/1.1" 200 None
10 - connectionpool  - _make_request   - http://localhost:None "GET /v1.35/containers/d8968bb48ff96244dc18416764d9d1111a6da3858e6c9d60c6de30c02b8dce10/json HTTP/1.1" 200 None
10 - connectionpool  - _make_request   - http://localhost:None "GET /v1.35/containers/25f83d7ea3320b60c03b96679cf38d7ecd99e96235cdd96efd7dccf23160e502/json HTTP/1.1" 200 None
10 - connectionpool  - _make_request   - http://localhost:None "GET /v1.35/images/6249560883cb6f7bd89a28de1b362dd867fe675c3767119e1a6fab0504b5036e/json HTTP/1.1" 200 None
10 - connectionpool  - _make_request   - http://localhost:None "GET /v1.35/images/4bb57d46fcd51993e5f903472f2c3c1f02a7d0eceef6dbddb16fcc591489641f/json HTTP/1.1

Docker container docker_aquarius_1 is running
	Tags: ['oceanprotocol/aquarius:v0.1.2']
Docker container docker_keeper-contracts_1 is running
	Tags: ['oceanprotocol/keeper-contracts:v0.3.12']
Docker container docker_mongodb_1 is running
	Tags: ['mongo:3.6']


In [7]:
def get_address(api_client, container_id,contract_name):

    # This is the python script to be executed in the running image
    python_script = r"import sys, json; print(json.load(open('/keeper-contracts/artifacts/{}.development.json', 'r'))['address'])".format(contract_name)

    # Wrap the script in quotes (string) and add the python shell command
    command = r"python -c " + '"' + python_script + '"'

    # Create and run the command
    ex = api_client.exec_create(container=container_id, cmd=command)

    return api_client.exec_start(ex)

# Get the docker image running the smart contracts
container_keeper_contracts = client.containers.get('docker_keeper-contracts_1')
addresses=dict()
addresses['market.address'] = get_address(low_level_api_client,container_keeper_contracts.id,'OceanMarket').decode("utf-8").rstrip()
addresses['auth.address'] = get_address(low_level_api_client,container_keeper_contracts.id,'OceanAuth').decode("utf-8").rstrip()
addresses['token.address'] = get_address(low_level_api_client,container_keeper_contracts.id,'OceanToken').decode("utf-8").rstrip()
addresses['didregistry.address'] = get_address(low_level_api_client,container_keeper_contracts.id,'DIDRegistry').decode("utf-8").rstrip()

print("Artifact addresses retrieved:")
pprint(addresses)

10 - connectionpool  - _make_request   - http://localhost:None "GET /v1.35/containers/docker_keeper-contracts_1/json HTTP/1.1" 200 None
10 - connectionpool  - _make_request   - http://localhost:None "POST /v1.35/containers/d8968bb48ff96244dc18416764d9d1111a6da3858e6c9d60c6de30c02b8dce10/exec HTTP/1.1" 201 74
10 - connectionpool  - _make_request   - http://localhost:None "POST /v1.35/exec/c05cd58aeb0da75bda38af1ed7b0bf28c8afc560461d3d74f0f9d5107ecb5b65/start HTTP/1.1" 101 0
10 - connectionpool  - _make_request   - http://localhost:None "POST /v1.35/containers/d8968bb48ff96244dc18416764d9d1111a6da3858e6c9d60c6de30c02b8dce10/exec HTTP/1.1" 201 74
10 - connectionpool  - _make_request   - http://localhost:None "POST /v1.35/exec/2d9b7c5be9afec756d636c057b7e826262114769b5fdee123060c2f9af70c056/start HTTP/1.1" 101 0
10 - connectionpool  - _make_request   - http://localhost:None "POST /v1.35/containers/d8968bb48ff96244dc18416764d9d1111a6da3858e6c9d60c6de30c02b8dce10/exec HTTP/1.1" 201 74
10 - c

Artifact addresses retrieved:
{'auth.address': '0x59d3631c86BbE35EF041872d502F218A39FBa150',
 'didregistry.address': '0xC89Ce4735882C9F0f0FE26686c53074E09B0D550',
 'market.address': '0x9561C133DD8580860B6b7E504bC5Aa500f0f06a7',
 'token.address': '0xCfEB869F69431e42cdB54A4F4f105C19C080A601'}
